In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [2]:
#dataEvents = pd.read_csv('/Users/jruipere/Dropbox (MIT)/Game-based Assessment/data_processing_scripts/data/anonymized_dataset.csv', sep=";")

In [168]:
dataEvents = pd.read_csv('/Users/pedroantonio/Desktop/data/anonymized_dataset.csv', sep=";")

In [169]:
levelsOfActivity = computeLevelsOfActivity2(dataEvents)

In [170]:
levelsOfActivity

,group,user,task_id,metric,value
0,95cd1f045a0329cf5595ae734ae7ae43,15246dfc4342ffe2601e4ecee9017eea,1. One Box,n_event,16.0
1,95cd1f045a0329cf5595ae734ae7ae43,15246dfc4342ffe2601e4ecee9017eea,2. Separated Boxes,n_event,18.0
2,95cd1f045a0329cf5595ae734ae7ae43,15246dfc4342ffe2601e4ecee9017eea,3. Rotate a Pyramid,n_event,26.0
3,95cd1f045a0329cf5595ae734ae7ae43,15246dfc4342ffe2601e4ecee9017eea,4. Match Silhouettes,n_event,9.0
4,95cd1f045a0329cf5595ae734ae7ae43,15246dfc4342ffe2601e4ecee9017eea,5. Removing Objects,n_event,24.0
...,...,...,...,...,...
991,95cd1f045a0329cf5595ae734ae7ae43,d1b81842e7f52c7e4bfee0b60735bdf1,Sugar Cones,redo_action,0.0
992,95cd1f045a0329cf5595ae734ae7ae43,d1b81842e7f52c7e4bfee0b60735bdf1,Tall and Small,redo_action,0.0
993,95cd1f045a0329cf5595ae734ae7ae43,d1b81842e7f52c7e4bfee0b60735bdf1,Unnecessary,redo_action,0.0
994,95cd1f045a0329cf5595ae734ae7ae43,d1b81842e7f52c7e4bfee0b60735bdf1,Warm Up,redo_action,0.0


In [ ]:
def computeLevelsOfActivity(dataEvents, group = 'all'):
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    
    #iterates in the groups and users of the data
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    
    activity_by_user = dataEvents.groupby(['group_user_id','group', 'user']).agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'})
    activity_by_user.index = activity_by_user['group_user_id'].values
    
    activity_by_user['active_time'] = np.nan
    activity_by_user['n_completed_puzzles'] = np.nan
    
    completedPuzzles = dict()   
    for user in activity_by_user['group_user_id'].unique():
        user_events = dataEvents[dataEvents['group_user_id'] == user]
        completedPuzzles[user] = set()

        # Computing active time
        previousEvent = None
        theresHoldActivity = 60 # np.percentile(allDifferences, 98) is 10 seconds
        activeTime = []

        for enum, event in user_events.iterrows():
            # If it is the first event
            if(previousEvent is None):
                previousEvent = event
                continue

            if(event['type'] == 'ws-puzzle_complete'):
                # adding key of the completed puzzle
                completedPuzzles[user].add(json.loads(event['data'])['task_id']) 

            delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
            if((delta_seconds < theresHoldActivity)):
                activeTime.append(delta_seconds)

            previousEvent = event

        activity_by_user.at[user, 'active_time'] = round(np.sum(activeTime)/60,2)
        activity_by_user.at[user, 'n_completed_puzzles'] = len(completedPuzzles[user])
        
    activity_by_user.drop(columns=['group_user_id'], inplace=True)
    activity_by_user = pd.melt(activity_by_user, id_vars=['group', 'user'], 
        value_vars=['n_events', 'n_different_events', 'active_time', 'n_completed_puzzles'], 
        var_name='metric', value_name='value')
    
    return activity_by_user

In [167]:
def computeLevelsOfActivity2(dataEvents, group = 'all'):
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    
    #iterates in the groups and users of the data
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['task_id'] = [json.loads(x)['task_id'] if 'task_id' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    dataEvents['group_user_task_id'] = dataEvents['group'] + '~' + dataEvents['user']+'~'+dataEvents['task_id']

         
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
          
    # the data is grouped by the necessary variables      
    activity_by_user = dataEvents.groupby(['group_user_id','group', 'user','group_user_task_id','task_id']).agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'}) 
    
    #indicate the index variable                                                                                                                                                               
    activity_by_user.index = activity_by_user['group_user_task_id'].values
                                                                                              
    #initialize the metrics                                                                                          
    activity_by_user['n_event'] = np.nan
    activity_by_user['n_different_events'] = np.nan
    activity_by_user['active_time'] = np.nan
    activity_by_user['snapshot'] = 0
    activity_by_user['paint'] = 0
    activity_by_user['rotate_view'] = 0
    activity_by_user['move_shape'] = 0
    activity_by_user['scale_shape'] = 0
    activity_by_user['create_shape'] = 0
    activity_by_user['delete_shape'] = 0
    activity_by_user['undo_action'] = 0
    activity_by_user['redo_action'] = 0
    
    #initialize the data structures
    userFunnelDict = dict()  
    puzzleEvents = dict()
    eventsDiff = []
    eventsDiff_puzzle = dict()
    timePuzzle = dict()
    globalTypesEvents = dict()
    typesEvents = dict()
    
    
    
    
    for user in dataEvents['group_user_id'].unique():
        
        # Computing active time
        previousEvent = None
        theresHoldActivity = 60 # np.percentile(allDifferences, 98) is 10 seconds
        activeTime = []
        
        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_puzzle_key = None

        for enum, event in user_events.iterrows():
            
            # If it is the first event
            if(previousEvent is None):
                previousEvent = event
                continue
            
            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):
                
                #create id: group+user+task_id                                                                              
                user_puzzle_key = event['group'] + '~' + event['user'] + '~' + json.loads(event['data'])['task_id']
                
                # initialize if the id is new                                                                              
                if(user_puzzle_key not in puzzleEvents.keys()):
                    puzzleEvents[user_puzzle_key]= 1
                    eventsDiff_puzzle[user_puzzle_key] = []
                    timePuzzle[user_puzzle_key] = 0
                    
                    globalTypesEvents[user_puzzle_key] = dict()
                    globalTypesEvents[user_puzzle_key]['ws-snapshot'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-paint'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-rotate_view'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-move_shape'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-scale_shape'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-create_shape'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-delete_shape'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-undo_action'] = 0
                    globalTypesEvents[user_puzzle_key]['ws-redo_action'] = 0
                    
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    
            # the event is not final event
            if(event['type'] not in ['ws-exit_to_menu', 'ws-puzzle_complete', 'ws-create_user']): 
                    puzzleEvents[user_puzzle_key] += 1
                                                                                              
                    #add the event type                                                                          
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    
                    #calculate the duration of the event                                                                          
                    delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                    if((delta_seconds < theresHoldActivity)):
                        timePuzzle[user_puzzle_key] += delta_seconds

                    previousEvent = event 
                    
                    #update event counters by type                                                                          
                    if(event['type'] == 'ws-snapshot'):
                        globalTypesEvents[user_puzzle_key]['ws-snapshot'] +=1
                    elif(event['type'] == 'ws-rotate_view'):
                        globalTypesEvents[user_puzzle_key]['ws-rotate_view'] +=1 
                    elif(event['type'] == 'ws-paint'):
                        globalTypesEvents[user_puzzle_key]['ws-paint'] +=1 
                    elif(event['type'] == 'ws-move_shape'):
                        globalTypesEvents[user_puzzle_key]['ws-move_shape'] +=1 
                    elif(event['type'] == 'ws-scale_shape'):
                        globalTypesEvents[user_puzzle_key]['ws-scale_shape'] +=1 
                    elif(event['type'] == 'ws-create_shape'):
                        globalTypesEvents[user_puzzle_key]['ws-create_shape'] +=1
                    elif(event['type'] == 'ws-delete_shape'):
                        globalTypesEvents[user_puzzle_key]['ws-delete_shape'] +=1
                    elif(event['type'] == 'ws-undo_action'):
                        globalTypesEvents[user_puzzle_key]['ws-undo_action'] +=1
                    elif(event['type'] == 'ws-redo_action'):
                        globalTypesEvents[user_puzzle_key]['ws-redo_action'] +=1    
                        
            # the puzzle ends        
            if(event['type'] in ['ws-exit_to_menu', 'ws-puzzle_complete']):
                                                                                              
                    #add the event type                                                                         
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    
                    #calculate the duration of the event                                                                          
                    delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                    if((delta_seconds < theresHoldActivity)):
                        timePuzzle[user_puzzle_key] += delta_seconds

                    previousEvent = event
                    
    # add the data by group_user_task_id            
    for i in dataEvents['group_user_task_id'].unique():
        key_split = i.split('~')
        if(key_split[2] != ''):
            activity_by_user.at[i, 'n_event'] = puzzleEvents[i]
            activity_by_user.at[i, 'n_different_events'] = len(set(eventsDiff_puzzle[i]))
            activity_by_user.at[i, 'active_time'] = timePuzzle[i]
            activity_by_user.at[i, 'snapshot'] = globalTypesEvents[i]['ws-snapshot']
            activity_by_user.at[i, 'paint'] = globalTypesEvents[i]['ws-paint']
            activity_by_user.at[i, 'rotate_view'] = globalTypesEvents[i]['ws-rotate_view']
            activity_by_user.at[i, 'move_shape'] = globalTypesEvents[i]['ws-move_shape']
            activity_by_user.at[i, 'scale_shape'] = globalTypesEvents[i]['ws-scale_shape']
            activity_by_user.at[i, 'create_shape'] = globalTypesEvents[i]['ws-create_shape']
            activity_by_user.at[i, 'delete_shape'] = globalTypesEvents[i]['ws-delete_shape']
            activity_by_user.at[i, 'undo_action'] = globalTypesEvents[i]['ws-undo_action']
            activity_by_user.at[i, 'redo_action'] = globalTypesEvents[i]['ws-redo_action']
            
    
    
    #delete row with NaN
    activity_by_user.dropna(inplace=True)
    #delete group_user_task_id column
    activity_by_user.drop(columns=['group_user_task_id'], inplace=True)
    
    #data output preparation                                                                                          
    activity_by_user = pd.melt(activity_by_user, id_vars=['group', 'user','task_id'], 
        value_vars=['n_event','n_different_events', 'active_time','snapshot','paint','rotate_view','move_shape','scale_shape','create_shape','delete_shape','undo_action','redo_action'], 
        var_name='metric', value_name='value')
        
    return activity_by_user

